In [2]:
import tensorflow as tf
from glob import glob
import os
import cv2
from PIL import Image
from tensorflow.keras.utils import normalize
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from mymodel import get_unet_model
from tensorflow.keras.optimizers import Adam

model = get_unet_model(256,256,3)
opt = Adam(learning_rate = 0.001)
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
print(type(model))

2022-01-27 14:57:56.565295: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-01-27 14:57:59.765966: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-01-27 14:57:59.775937: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-01-27 14:58:00.103700: E tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:927] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-01-27 14:58:00.103772: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:07:00.0 name: NVIDIA GeForce RTX 2060 SUPER computeCapability: 7.5
coreClock: 1.695GHz coreCount: 34 deviceMemorySize: 8.00GiB deviceMemoryBandwidth: 417.29GiB/s
2022-01-27 14:58:00.103790: I tensorflow/stream_executor/

<class 'tensorflow.python.keras.engine.functional.Functional'>


In [3]:
aug = ImageDataGenerator(
	rotation_range=180,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")
total = 0


In [4]:
from sklearn.model_selection import train_test_split

img_dataset = []
msk_dataset = []


f_img = sorted(glob('dataset/train/asagao/*'))
f_msk = sorted(glob('dataset/validation/asagao/*'))
for img, msk in zip(f_img, f_msk):
    mask = cv2.imread(msk, cv2.IMREAD_GRAYSCALE)
    image = cv2.imread(img)

    image = image.astype('float32') / 255.0
    (thresh, im_bw) = cv2.threshold(mask, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    mask = cv2.threshold(mask, thresh, 255, cv2.THRESH_BINARY)[1]
    if mask.shape[0] != 256 or mask.shape[1] != 256:
        print(msk, 'was invalid size')
        continue
    # image = image.resize((SIZE, SIZE))

    img_dataset.append(np.array(image))
    msk_dataset.append(np.array(mask))


f_img = sorted(glob('dataset/train/not_asagao/*'))
f_msk = sorted(glob('dataset/validation/not_asagao/*'))
print(len(f_img), len(f_msk))
for img, msk in zip(f_img, f_msk):
    mask = cv2.imread(msk, cv2.IMREAD_GRAYSCALE)
    image = cv2.imread(img)

    image = image.astype('float32') / 255.0
    (thresh, im_bw) = cv2.threshold(mask, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    mask = cv2.threshold(mask, thresh, 255, cv2.THRESH_BINARY)[1]
    if mask.shape[0] != 256 or mask.shape[1] != 256:
        print(msk, 'was invalid size')
        continue
    # image = image.resize((SIZE, SIZE))

    img_dataset.append(np.array(image))
    msk_dataset.append(np.array(mask))


asagao_dataset = np.array(img_dataset)
asagao_msk_dataset = np.expand_dims(np.array(msk_dataset) /255., -1)
X_train, X_test, y_train, y_test = train_test_split(asagao_dataset, asagao_msk_dataset, test_size= 0.10, random_state=0)


del f_img, f_msk, image, im_bw, thresh, mask, msk, img

dataset/validation/asagao/asagao_mask_0066.png was invalid size
dataset/validation/asagao/asagao_mask_0082.png was invalid size
4554 4554


In [5]:
history = model.fit(x=aug.flow(X_train, y_train, batch_size=8),
                    verbose = 1,
                    batch_size = 8,
                    epochs = 100,
                    validation_data = (X_test, y_test),
                    shuffle = False)
model.save("50m_30m_10mUnet_All")

2022-01-27 15:03:31.449771: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-01-27 15:03:31.462999: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3393735000 Hz


Epoch 1/100


2022-01-27 15:03:36.068844: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7
2022-01-27 15:04:02.300294: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10


542/542 [==============================] - ETA: 0s - loss: 0.2422 - accuracy: 0.9510